Agent VG multi record producers

In [18]:
# LIBS
import requests
import os
import json
import time
from dotenv import load_dotenv
import openai

In [12]:
# LOAD env vars
load_dotenv()

True

In [13]:
# REST API (ServiceNow) URL and credentials
instance = os.getenv('em_lab_instance')
user = os.getenv('em_lab_u_name')
pwd = os.getenv('em_lab_p_word')
url = f"https://{instance}.service-now.com/api/now/table/incident"

# HEADERS
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

In [14]:
# PING (testing connection)
ping_response = requests.get(url, auth=(user, pwd), headers=headers)
print(ping_response.status_code)

200


In [15]:
# PAYLOAD GENERATOR
#### Add AI here and payload generator
# could add a humor mode and a serious mode???****

In [20]:
# setup openai api key
openai.api_key = os.getenv('Matts_OpenAI_KEY')

In [ ]:
def generate_demo_payload():
    try:
        prompt = """
        Generate a JSON object for a ServiceNow incident record with the following fields:
        1. short_description: A brief description of the incident.
        2. description: A detailed description of the incident.
        3. urgency: A numeric value between 1 and 3, where 1 is high urgency.
        4. impact: A numeric value between 1 and 3, where 1 is high impact.
        Provide values that could realistically represent IT-related incidents.
        """
        
        response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=prompt,
            max_tokens=100,
            n=1,
            stop=None,
            temperature=0.7
        )
        
        # Parse the response to get the JSON object
        payload = json.loads(response.choices[0].text.strip())
        return payload
    
    except Exception as e:
        print(f"An error occurred while generating payload: {e}")
        return None

In [ ]:
# TEST, generate and view the payload
payload_test = generate_demo_payload()
print(json.dumps(payload_test, indent=2))


In [ ]:
# need to add caller and other fields you want

In [16]:
# PAYLOADS (temp)
payloads = [
    {
        "short_description": "Computer is overheating",
        "description": "Computer is overheating, span running for a while.",
        "caller_id": "Employee"

    },
    {
        "short_description": "Issue accessing VPN",
        "description": "I have problems signing onto the VPN after using it for DraftKings",
        "caller_id": "Employee"

    },
    {
        "short_description": "Wifi connectivity issue in the office",
        "description": "When I try to connect to the office wifi, it doesn't connect.",
        "caller_id": "Employee"

    },
    {
        "short_description": "Printer not working",
        "description": "When I try to print, the printer doesn't respond.",
        "caller_id": "Employee"

    }
]

In [17]:
# CREATE INCIDENTs
def create_incident(payload):
    try:
        response = requests.post(url, auth=(user, pwd), headers=headers, data=json.dumps(payload))
        # Check if the response was successful
        if response.status_code == 201:
            print(f"Incident created successfully: {json.loads(response.text)['result']['number']}")
        else:
            print(f"Failed to create incident: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"An error occurred: {e}")

# intevals (simlated INCs arrival)
intervals = [5, 10, 15]  

# loop through each payload
for i, payload in enumerate(payloads):
    create_incident(payload)  # create the incident
    if i < len(intervals):  # if it's not the last one, wait for the next interval
        time.sleep(intervals[i])  # wait before sending the next request

Incident created successfully: INC0010002
Incident created successfully: INC0010003
Incident created successfully: INC0010004
Incident created successfully: INC0010005


| END OF PROGRAM |
|-|